# Collaborative Filtering Methods
In this notebook we will tests various algorithms that will help us implement **Latent Factors - Collaborative Fileting**. We will test the methods of **Alternating Least Squares Matrix Factorization (ALS)** and **Simultaneous Least Squares Matrix Factorization (SLS)**. Along with these methods, we will also test **Gradient Descent** and **Line Search**, two algorithms that will help us implement ALS and SLS.

For ALS we use two objective functions $J_u$ and $J_a$ which we define as.
$$
 J_u(\Theta) = \frac{1}{2} \sum_{i=1}^{m} \left[ \sum_{\{j\:|\:1 \le j \le n,\: r(i,j) = 1\}} ((\theta^{(i)})^\mathsf{T}x^{(j)} - y(i,j))^2 \right] \text{. }
$$ and


In [1]:
# data analysis and wrangling
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt

# functions to test
from gradientDescent import gradientDescent
from alternatingLeastSquares import ALS
from lineSearchAlgorithm import lineSearch
from cost_functions import get_Ju_and_DJu, get_Ja_and_DJa
from aux_functions import initializeTheta, initializeX

In [2]:
tol = 1e-3
max_iter = 100

## Sample Data
We create a simple $5\times4$ interactions matrix to test our algorithms.

In [8]:
R = np.array(
    [
        [5, 1, np.NAN, 4],
        [np.NAN, 9, np.NAN, 2],
        [9, 2, 1, np.NAN],
        [6, np.NAN, 7, 4],
        [np.NAN, 4, 7, 5],
    ]
)

m, n = R.shape

We define the range $f$ for the matrices $Q$ and $P$ that will approximate $R$.

In [9]:
f = 4

We initialize $Q$ and $P$ with values from a standard normal random variable keeping a constant seed for consistent results.

In [ ]:
# Seed
np.random.seed(43)

# Q and P initilization
Q = initializeTheta(R.shape[0], f)
P = initializeX(R.shape[1], f)

## Line Search Test
The **Line Search** algorithm return a step that meets the **Wolfe** conditions. Here, we test if the implmentation is correct.

First, we define $Ju$, our objective function to  given $R$, $P$, and $f$.

In [6]:
Ju, DJu = get_Ju_and_DJu(R, P, p)
Theta = Theta.flatten()
p_k = -DJu(Theta)
alpha = lineSearch(Ju, DJu, Theta, p_k)
alpha

0.25

In [ ]:
Ju, DJu = get_Ju_and_DJu(R, X, p)

Theta = Theta.flatten()
newThetaResults = gradientDescent(Ju, Theta, DJu, tol, max_iter)
newTheta = newThetaResults["parameters"].reshape(R.shape[0], p)
Ju_values = newThetaResults["func_values"]

plt.plot(Ju_values)
plt.yscale("log")
plt.show()

In [ ]:
alpha_values = newThetaResults["alpha_values"]
alpha_values

In [ ]:
newR = newTheta @ X.T

In [ ]:
pd.DataFrame(np.around(newR, 2))

In [ ]:
pd.DataFrame(R)

In [ ]:
np.random.seed(0)
Theta = initializeTheta(R.shape[0], p)
X = initializeX(R.shape[1], p)

In [ ]:
Ja, DJa = get_Ja_and_DJa(R, Theta, p)
X = X.flatten()
p_k = -DJa(X)
alpha = lineSearch(Ja, DJa, X, p_k)
alpha

In [ ]:
Ja, DJa = get_Ja_and_DJa(R, Theta, p)

X = X.flatten()
newXResults = gradientDescent(Ja, X, DJa,tol, max_iter)
newX = newXResults["parameters"].reshape(R.shape[1], p)
Ja_values = newXResults["func_values"]

plt.plot(Ja_values)
plt.yscale("log")
plt.show()

In [ ]:
alpha_values = newXResults["alpha_values"]
alpha_values

In [ ]:
Ja_values

In [ ]:
newR = Theta @ newX.T

In [ ]:
pd.DataFrame(np.around(newR, 2))

In [ ]:
pd.DataFrame(R)

In [ ]:
np.random.seed(0)
als_result = ALS(R, p, alpha, 0, tol, 100, max_iter)

In [ ]:
newTheta = als_result["Theta"]
newX = als_result["X"]
J_values = als_result["J_values"]

In [ ]:
plt.plot(J_values)
plt.show()

In [ ]:
plt.plot(J_values)
plt.yscale("log")
plt.show()

In [ ]:
newR = newTheta @ newX.T

In [ ]:
pd.DataFrame(np.around(newR, 1))

In [ ]:
pd.DataFrame(R)